# Time evolution of 1-D Schrodinger Equation with wavelets
## Cross correlation coefficients

In [1]:
import sys, struct
import numpy as np
import matplotlib.pyplot as plt
import scipy
from copy import deepcopy
from array import array

Here we calculate coefficients $C_{jp}^{2k}$ standing in the following expression
defining the time evolution operator.
Our main operator has the following expansion
\begin{equation}
\label{operator_correlation_expansion}
    \left[ \sigma_l^{\mathfrak n} \right]_{pj}
    (a)
    =
    \sum_{k = 0}^{\infty}
    C_{jp}^{2k}
    \widetilde J_{2k + j + p}(l, a)
    ,
\end{equation}
where $a = t \mathfrak N^2 = t 4^{\mathfrak n}$
and
\begin{equation}
\label{power_integral}
    \widetilde J_m
    =
    \frac
    {
        I_m
        e^{ i \frac {\pi}4 (m - 1) }
    }
    {
        2 \pi ( m + 2 )!
    }
    =
    \frac
    {
        e^{ i \frac {\pi}4 (m - 1) }
    }
    {
        2 \pi ( m + 2 )!
    }
    \int_{\mathbb R}
    \exp
    \left(
        \rho l \exp \left( i \frac \pi 4 \right) - a \rho^2
    \right)
    \rho^m
    d \rho
\end{equation}


\begin{equation*}
\begin{aligned}
    \widetilde A_0^{j+1}
    &=
    \sqrt{ \frac{ 2j + 3 }{ 2j - 1 } }
    \widetilde A_0^{j-1}
    \\
    \widetilde A_1^{j+1}
    &=
    \sqrt{ \frac{ 2j + 3 }{ 2j - 1 } }
    \frac{ j - 1 }{ j + 1 }
    \widetilde A_1^{j-1}
    -
    \sqrt{ \frac {(2j + 1)(2j + 3)}{(2j + 2)(2j + 2)} }
    \widetilde A_0^j
    \\
    \widetilde A_2^{j+1}
    &=
    \sqrt{ \frac{ 2j + 3 }{ 2j - 1 } }
    \left( \frac{ j - 1 }{ j + 1 } \right)^2
    \widetilde A_2^{j-1}
    -
    \sqrt{ \frac {(2j + 1)(2j + 3)}{(2j + 2)(2j + 2)} }
    \frac j{ j + 1 }
    \widetilde A_1^j
    \\
    &\ldots \qquad \ldots \qquad \ldots \qquad \ldots \qquad \ldots
    \\
    \widetilde A_{j-1}^{j+1}
    &=
    \sqrt{ \frac{ 2j + 3 }{ 2j - 1 } }
    \left( \frac{ j - 1 }{ j + 1 } \right)^{j-1}
    \widetilde A_{j-1}^{j-1}
    -
    \sqrt{ \frac {(2j + 1)(2j + 3)}{(2j + 2)(2j + 2)} }
    \left( \frac j{ j + 1 } \right)^{j-2}
    \widetilde A_{j-2}^j
    \\
    \widetilde A_j^{j+1}
    &=
    -
    \sqrt{ \frac {(2j + 1)(2j + 3)}{(2j + 2)(2j + 2)} }
    \left( \frac j{ j + 1 } \right)^{j-1}
    \widetilde A_{j-1}^j
    \\
    \widetilde A_{j+1}^{j+1}
    &=
    -
    \sqrt{ \frac {(2j + 1)(2j + 3)}{(2j + 2)(2j + 2)} }
    \left( \frac j{ j + 1 } \right)^j
    \widetilde A_j^j
\end{aligned}
\end{equation*}

The following function generates numpay array $\widetilde A^{j+1}$
for given arrays $A = \widetilde A^{j-1}$, $B = \widetilde A^j$,
using the above reccurence relation.

In [2]:
def tilde_A_transformation(A, B):
    j = len(A)
    tilde_A = np.ones_like(A)
    q = ( j - 1. )/( j + 1. )
    for i in range(1, j):
        tilde_A[i] = tilde_A[i-1] * q
    tilde_B = np.ones_like(B)
    q = j /( j + 1. )
    for i in range(1, j + 1):
        tilde_B[i] = tilde_B[i-1] * q
    
    alpha = np.sqrt( (2.0 * j + 3) / (2.0 * j - 1) )
    beta = - np.sqrt( (2.0 * j + 3) / (2.0 * j + 2) * (2.0 * j + 1) / (2.0 * j + 2) )
    res = np.zeros( len(B) + 1 )    #j + 2
    res[:-2] = res[:-2] + alpha * A * tilde_A
    res[1:] = res[1:] + beta * B * tilde_B
    return res

Using the above transformation, that is exactly the same for $A$- and $B$-coefficients, and knowing the first coefficients (given below), one can calculate the correlation coefficients as follows.

\begin{equation*}
    \widetilde A_0^0 = -1
    , \quad
    \widetilde B_0^0 = 1
    ,
\end{equation*}
\begin{equation*}
    \widetilde A_0^1 = \sqrt{3}
    , \quad
    \widetilde A_1^1 = \frac{\sqrt{3}}2
    , \quad
    \widetilde B_0^1 = \sqrt{3}
    , \quad
    \widetilde B_1^1 = - \frac{\sqrt{3}}2
    .
\end{equation*}

\begin{equation*}
    C_{jp}^k
    =
    \sum_{m=0}^j
    \sum_{q=0}^p
    \frac{ (-1)^{m+1} (k + 2 + j + p)! (4j)^m (4p)^q }{ (k + 2 + j + p + m + q)! }
    \left(
        \widetilde A_m^j \widetilde B_q^p
        +
        (-1)^{k + j + p + m + q}
        \widetilde B_m^j \widetilde A_q^p
    \right)
    ,
\end{equation*}

In [3]:
def get_correlation_coefficient(k, j, p):
    A = get_first_matrix(k, j, p)
    B = get_second_matrix(k, j, p)
    return np.sum(A * B)

def get_first_matrix(k, j, p):
    F = np.empty( (j + 1, p + 1) )
    F[0,0] = -1
    K = k + 2 + j + p
    for m in range(1, j + 1):
        F[m, 0] = F[m - 1, 0] * (-4 * j) / (K + m)
    for m in range(0, j + 1):
        for q in range(1, p + 1):
            F[m, q] = F[m, q - 1] * (4 * p) / (K + m + q) 
    return F

def get_second_matrix(k, j, p):
    A0 = np.array( [-1.0] )
    A1 = np.array( [np.sqrt(3), 0.5 * np.sqrt(3)] )
    B0 = np.array( [1.0] )
    B1 = np.array( [np.sqrt(3), -0.5 * np.sqrt(3)] )
    A = [ A0, A1 ]
    B = [ B0, B1 ]
    for i in range( max(j,p) - 1 ):
        A.append( tilde_A_transformation(A[-2], A[-1]) )
        B.append( tilde_A_transformation(B[-2], B[-1]) )
    F = np.empty( (j + 1, p + 1) )
    K = k + j + p
    for m in range(0, j + 1):
        for q in range(0, p + 1):
            F[m, q] = A[j][m] * B[p][q] + (-1)**(K + m + q) * B[j][m] * A[p][q]
    return F

The amount of matrices is $K$ and the size of each matrix is $\mbox{Legendre_order} \times \mbox{Legendre_order}$.

In [4]:
K = 60
Legendre_order = 40

C_even = []

for k in range(0, 2*K, 2):
    C = np.ones((Legendre_order, Legendre_order))
    for j in range(Legendre_order):
        for p in range(Legendre_order):
            C[j,p] = get_correlation_coefficient(k, j, p)
    C_even.append(C)

C_even = np.array(C_even)

In [5]:
print(C_even[0])
print(C_even[1])

[[ 2.00000000e+00  1.73205081e+00  8.94427191e-01 ... -6.36960146e-05
   7.04987008e-05 -2.44140625e-04]
 [-1.73205081e+00 -2.00000000e+00 -1.29099445e+00 ...  1.09083650e-04
  -1.40991864e-04  3.66210938e-04]
 [ 8.94427191e-01  1.29099445e+00  1.00000000e+00 ...  7.96109463e-08
   3.28887739e-04 -1.22070312e-04]
 ...
 [ 6.36960146e-05  6.10351557e-05  9.14731234e-05 ...  2.23854212e-02
   2.13240087e-02 -1.02078989e-02]
 [ 7.04987008e-05 -1.50218769e-04  2.59477221e-04 ... -1.39160156e-02
   2.92663574e-02 -1.92871094e-02]
 [ 2.44140625e-04  4.88281250e-04  1.22070312e-04 ...  1.86862232e-02
   2.70202501e-02 -1.26493813e-01]]
[[ 2.00000000e+00  2.30940108e+00  1.59719141e+00 ... -5.65907154e-05
   5.05571417e-05 -1.22070312e-04]
 [-2.30940108e+00 -3.00000000e+00 -2.32379001e+00 ...  7.08562303e-05
  -7.65058334e-05  4.88281250e-04]
 [ 1.59719141e+00  2.32379001e+00  2.00000000e+00 ... -7.09606968e-05
   1.77654086e-04 -4.57763672e-04]
 ...
 [ 5.65907154e-05  5.77059658e-05  2.5184329

One prepares the data for writing to a file in the following order:
1) size of documentation text;
2) documentation text;
3) the amount of matrices;
4) the amount of columns (order) in each matrix;
5) the first matrix stored row after row, the second one, and so on.

In [6]:
documentation_text = "Time evolution Schrodinger cross correlation coefficients C_{p,j}^{2k} corresponding to "
documentation_text += "Legendre scaling basis with k = 0, ..., K - 1 and p,j = 0, ..., L - 1. "
documentation_text += "Here the amount of matrices K = "
documentation_text += str(K)
documentation_text += " and the Legendre order L = "
documentation_text += str(Legendre_order)
documentation_text += ". After this documentation text follows interger for K, then interger for L. "
documentation_text += "Then follow coefficients C_{0,0}^{0}, C_{0,1}^{0}, ..., C_{0,L-1}^{0}, C_{1,0}^{0},"
documentation_text += " C_{1,1}^{0}, ..., C_{1,L-1}^{0}, ..., C_{L-1,L-1}^{2(K-1)}."

text_lenght = len(documentation_text)
text_lenght = np.array(text_lenght, dtype=np.int32)
documentation_text_array = array('u', list(documentation_text))
K_array = np.array(K, dtype=np.int32)
Legendre_order_array = np.array(Legendre_order, dtype=np.int32)

## Location should be defined here.

In [7]:
#lib = "/Users/edi011/Documents/Projects/Project 01./binary file utility"
#lib = "/Users/edi011/mrcpp/pilot"
#location = lib + "/test_file.bin"

lib = "../share/mwfilters"
location = lib + "/Schrodinger_evolution_cross_correlation_coefficients_Legendre_scaling_type.bin"

## We write into a file the matrices $C_{jp}^{2k}$ for each $k = 0, \ldots, K - 1$. 

In [8]:
output_file = open(location, 'wb')

text_lenght.tofile(output_file)
documentation_text_array.tofile(output_file)
K_array.tofile(output_file)
Legendre_order_array.tofile(output_file)
C_even.tofile(output_file)

output_file.close

<function BufferedWriter.close>

## By default Eigen library reads from a file the transpose matrices $C_{pj}^{2k} = \left(C_{jp}^{2k}\right)^T$ for each $k = 0, \ldots, K - 1$. This is convinient for further calculations.